In [40]:
!pip install python-box

In [41]:
import numpy as np
import yaml
from box import Box

import torch
import torch.nn as nn
import torch.optim as optim

import simmim
from swin_v2 import SwinTransformerV2

from torch.cuda.amp import autocast, GradScaler
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm
import time

from timm.data import Mixup
import transformers

In [42]:
simmim_config = yaml.load(open('config/pretrain.yaml'), Loader=yaml.FullLoader)
simmim_config

{'MODEL': {'TYPE': 'swinv2',
  'NAME': 'simmim_pretrain',
  'DROP_PATH_RATE': 0.0,
  'SIMMIM': {'NORM_TARGET': {'ENABLE': True, 'PATCH_SIZE': 47}},
  'SWIN': {'EMBED_DIM': 96,
   'DEPTHS': [2, 2, 6, 2],
   'NUM_HEADS': [3, 6, 12, 24],
   'WINDOW_SIZE': 6,
   'PATCH_SIZE': 4}},
 'DATA': {'IMG_SIZE': 192,
  'MASK_PATCH_SIZE': 32,
  'MASK_RATIO': 0.6,
  'BATCH_SIZE': 1024,
  'NUM_WORKERS': 24,
  'DATA_PATH': '../../data/sports'},
 'TRAIN': {'EPOCHS': 100,
  'WARMUP_EPOCHS': 10,
  'BASE_LR': 0.0014,
  'WEIGHT_DECAY': 0.05,
  'CLIP_GRAD': 5},
 'TAG': 'simmim_pretrain__swinv2_tiny__img192_window6__800ep'}

In [43]:
encoder_config = {'img_size':simmim_config['DATA']['IMG_SIZE'], 
                'patch_size':simmim_config['MODEL']['SWIN']['PATCH_SIZE'], 
                'in_chans':3, 
                'num_classes':100,
                'embed_dim':simmim_config['MODEL']['SWIN']['EMBED_DIM'], 
                'depths':simmim_config['MODEL']['SWIN']['DEPTHS'], 
                'num_heads':simmim_config['MODEL']['SWIN']['NUM_HEADS'],           
                'window_size':simmim_config['MODEL']['SWIN']['WINDOW_SIZE'], 
                'mlp_ratio':4., 
                'qkv_bias':True, 
                'qk_scale':None,
                'drop_rate':0., 
                'attn_drop_rate':0., 
                'drop_path_rate':simmim_config['MODEL']['DROP_PATH_RATE'],
                'norm_layer':nn.LayerNorm, 
                'patch_norm':True, 
                'pretrained_window_sizes':[0,0,0,0],
                'ape':True}

encoder_stride = 32
in_chans = encoder_config['in_chans']
patch_size = encoder_config['patch_size']

In [44]:
encoder = simmim.SwinTransformerV2ForSimMIM(**encoder_config)

In [45]:
model = simmim.SimMIM( encoder=encoder, 
                       encoder_stride=encoder_stride, 
                       in_chans=in_chans, 
                       patch_size=patch_size)

In [46]:
mask_generator = simmim.MaskGenerator(input_size=224,
                                      mask_patch_size=28,
                                      model_patch_size=28,
                                      mask_ratio=.6)
mask = mask_generator()
mask

array([[0, 1, 0, 0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 1, 1],
       [0, 0, 1, 1, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 1, 0],
       [1, 0, 0, 0, 1, 1, 1, 0],
       [1, 0, 1, 0, 0, 1, 1, 1],
       [1, 0, 1, 1, 1, 0, 0, 1],
       [1, 1, 0, 1, 1, 1, 0, 1]])

In [47]:
print(f"생성된 mask의 비율은 {mask.sum() / (mask.shape[0]*mask.shape[1])*100}%")

생성된 mask의 비율은 60.9375%


In [48]:
simmim_config = Box(simmim_config)
dataloader = simmim.build_loader_simmim(simmim_config)

samples = next(iter(dataloader))
len(samples)

3

In [49]:
samples[0].shape, samples[1].shape, samples[2].shape 

(torch.Size([1024, 3, 192, 192]),
 torch.Size([1024, 48, 48]),
 torch.Size([1024]))

In [50]:
base_lr = float(simmim_config.TRAIN.BASE_LR)
weight_decay = simmim_config.TRAIN.WEIGHT_DECAY
optimizer = optim.AdamW(model.parameters(), lr=base_lr, weight_decay=weight_decay)
warmup_epochs = simmim_config.TRAIN.WARMUP_EPOCHS
train_epochs = simmim_config.TRAIN.EPOCHS

scheduler = transformers.get_cosine_schedule_with_warmup(optimizer, 
                                                        num_warmup_steps=warmup_epochs*len(dataloader), 
                                                        num_training_steps=train_epochs*len(dataloader),
                                                        num_cycles=0.5)

2024-01-15 14:26:47.748616: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-15 14:26:47.748686: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-15 14:26:47.749528: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-15 14:26:47.755498: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-15 14:26:48.650772: W tensorflow/compiler/tf2

In [51]:
device = 'cuda:3'
model.to(device)
torch.backends.cudnn.benchmark = True

model_save = True
simmim_path = '../../models/swin2/simmim.pth'

In [52]:
training_time = 0
losses = []
val_losses = []
lrs = []
best_loss = float('inf')

# GradScaler 초기화
scaler = GradScaler()

for epoch in range(train_epochs):
    model.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Epoch {epoch + 1}")
    
    for _, data in pbar:
        image, mask = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()

        # AutoCast 적용
        with autocast():
            loss = model(image, mask)
            
        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer)
        if simmim_config.TRAIN.CLIP_GRAD:
            clip_grad_norm_(model.parameters(), max_norm=simmim_config.TRAIN.CLIP_GRAD)
        else:
            clip_grad_norm_(model.parameters())

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
            
        lr = optimizer.param_groups[0]["lr"]
        lrs.append(lr)
        running_loss += loss.item()

    epoch_loss = running_loss / len(dataloader)
    losses.append(epoch_loss)

    # 모델 저장
    if epoch_loss < best_loss:
        
        best_loss = epoch_loss
        vit_save = model_save
        if vit_save:
            torch.save(model.state_dict(), simmim_path)
        
    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss:.4f}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
    
    if vit_save:
        text += f' - model saved!'
        vit_save = False    
        
    print(text)


Epoch 1: 100%|██████████| 14/14 [00:19<00:00,  1.41s/it]


	Loss: 1.1431, LR: 0.00014000000000000001, Duration: 20.80 sec - model saved!


Epoch 2: 100%|██████████| 14/14 [00:16<00:00,  1.19s/it]


	Loss: 1.0948, LR: 0.00028000000000000003, Duration: 18.31 sec - model saved!


Epoch 3: 100%|██████████| 14/14 [00:17<00:00,  1.24s/it]


...

	Loss: 0.5173, LR: 1.7051648181230617e-06, Duration: 17.74 sec



Epoch 99: 100%|██████████| 14/14 [00:17<00:00,  1.25s/it]

	Loss: 0.5166, LR: 4.264210866329665e-07, Duration: 18.66 sec



Epoch 100: 100%|██████████| 14/14 [00:17<00:00,  1.24s/it]

	Loss: 0.5177, LR: 0.0, Duration: 18.38 sec


In [ ]:
model.cpu()
del model
torch.cuda.empty_cache()

In [147]:
model = SwinTransformerV2(pretrained_window_sizes=[6,6,6,6], ape=True)
model.state_dict().keys()

odict_keys(['absolute_pos_embed', 'embeddings.patch_embeddings.weight', 'embeddings.patch_embeddings.bias', 'embeddings.norm.weight', 'embeddings.norm.bias', 'stages.0.blocks.0.attn_mask', 'stages.0.blocks.0.attn.t_scale', 'stages.0.blocks.0.attn.relative_coords_table', 'stages.0.blocks.0.attn.relative_position_index', 'stages.0.blocks.0.attn.crpb_mlp.0.weight', 'stages.0.blocks.0.attn.crpb_mlp.0.bias', 'stages.0.blocks.0.attn.crpb_mlp.3.weight', 'stages.0.blocks.0.attn.qkv.weight', 'stages.0.blocks.0.attn.qkv.bias', 'stages.0.blocks.0.attn.proj.weight', 'stages.0.blocks.0.attn.proj.bias', 'stages.0.blocks.0.norm1.weight', 'stages.0.blocks.0.norm1.bias', 'stages.0.blocks.0.mlp.fc1.weight', 'stages.0.blocks.0.mlp.fc1.bias', 'stages.0.blocks.0.mlp.fc2.weight', 'stages.0.blocks.0.mlp.fc2.bias', 'stages.0.blocks.0.norm2.weight', 'stages.0.blocks.0.norm2.bias', 'stages.0.blocks.1.attn_mask', 'stages.0.blocks.1.attn.t_scale', 'stages.0.blocks.1.attn.relative_coords_table', 'stages.0.blocks.1

In [148]:
model.embeddings.patch_embeddings.weight[0]

tensor([[[-8.8096e-02,  1.7407e-03, -9.6400e-02,  7.7608e-02],
         [-5.7258e-02, -1.4746e-02, -4.8214e-02, -4.3813e-03],
         [-1.1506e-01,  6.1205e-02, -9.6893e-02,  5.8575e-02],
         [ 8.7466e-02,  9.3681e-02, -6.0156e-02,  1.2314e-01]],

        [[ 2.6655e-02, -1.3461e-01,  1.3593e-01,  1.2407e-01],
         [ 2.6047e-02,  7.9218e-05,  1.1690e-01,  1.2052e-01],
         [ 1.2878e-01,  6.8909e-02,  2.3261e-02,  7.0877e-02],
         [ 3.6387e-02,  5.2186e-02,  3.7446e-04,  6.4454e-02]],

        [[ 6.4631e-02, -1.1809e-01,  4.7566e-02, -1.2559e-01],
         [ 1.2881e-01, -6.7343e-02,  1.4365e-01, -4.8204e-02],
         [-9.8623e-02, -5.9706e-02,  8.2210e-03, -6.7486e-02],
         [-5.2235e-03,  9.1597e-02, -1.1536e-01, -1.5746e-02]]],
       grad_fn=<SelectBackward0>)

In [149]:
swin_config = yaml.load(open('config/train.yaml'), Loader=yaml.FullLoader)
swin_config

{'MODEL': {'TYPE': 'swinv2',
  'NAME': 'simmim_train',
  'PRETRAINED': '../../models/swin2/simmim.pth',
  'DROP_PATH_RATE': 0.2,
  'SWIN': {'EMBED_DIM': 96,
   'DEPTHS': [2, 2, 6, 2],
   'NUM_HEADS': [3, 6, 12, 24],
   'WINDOW_SIZE': 7,
   'PATCH_SIZE': 4}},
 'DATA': {'IMG_SIZE': 224,
  'MASK_PATCH_SIZE': 32,
  'MASK_RATIO': 0.6,
  'BATCH_SIZE': 960,
  'NUM_WORKERS': 24,
  'DATA_PATH': '../../data/sports'},
 'TRAIN': {'EPOCHS': 20,
  'WARMUP_EPOCHS': 10,
  'BASE_LR': '1e-4',
  'WEIGHT_DECAY': 0.05,
  'CLIP_GRAD': 5}}

In [150]:
def load_pretrained(config, model):
    print(f"==============> Loading weight {config.MODEL.PRETRAINED} for fine-tuning......")
    state_dict = torch.load(config.MODEL.PRETRAINED, map_location='cpu')

    # remain encoder only
    not_encoder_keys = [k for k in state_dict.keys() if 'encoder' not in k]
    for k in not_encoder_keys:
        del state_dict[k]
        
    # remove prefix encoder.
    state_dict = {k.replace('encoder.', ''):v for k, v in state_dict.items()}

    # delete relative_position_index since we always re-init it
    relative_position_index_keys = [k for k in state_dict.keys() if "relative_position_index" in k]
    for k in relative_position_index_keys:
        del state_dict[k]

    # delete relative_coords_table since we always re-init it
    relative_position_index_keys = [k for k in state_dict.keys() if "relative_coords_table" in k]
    for k in relative_position_index_keys:
        del state_dict[k]

    # delete attn_mask since we always re-init it
    attn_mask_keys = [k for k in state_dict.keys() if "attn_mask" in k]
    for k in attn_mask_keys:
        del state_dict[k]

    # bicubic interpolate relative_position_bias_table if not match
    relative_position_bias_table_keys = [k for k in state_dict.keys() if "relative_position_bias_table" in k]
    for k in relative_position_bias_table_keys:
        relative_position_bias_table_pretrained = state_dict[k]
        relative_position_bias_table_current = model.state_dict()[k]
        L1, nH1 = relative_position_bias_table_pretrained.size()
        L2, nH2 = relative_position_bias_table_current.size()
        if nH1 != nH2:
            print(f"Error in loading {k}, passing......")
        else:
            if L1 != L2:
                # bicubic interpolate relative_position_bias_table if not match
                S1 = int(L1 ** 0.5)
                S2 = int(L2 ** 0.5)
                relative_position_bias_table_pretrained_resized = torch.nn.functional.interpolate(
                    relative_position_bias_table_pretrained.permute(1, 0).view(1, nH1, S1, S1), size=(S2, S2),
                    mode='bicubic')
                state_dict[k] = relative_position_bias_table_pretrained_resized.view(nH2, L2).permute(1, 0)

    # bicubic interpolate absolute_pos_embed if not match
    absolute_pos_embed_keys = [k for k in state_dict.keys() if "absolute_pos_embed" in k]
    for k in absolute_pos_embed_keys:
        # dpe
        absolute_pos_embed_pretrained = state_dict[k]
        absolute_pos_embed_current = model.state_dict()[k.replace('encoder.','')]
        _, L1, C1 = absolute_pos_embed_pretrained.size()
        _, L2, C2 = absolute_pos_embed_current.size()
        if C1 != C1:
            print(f"Error in loading {k}, passing......")
        else:
            if L1 != L2:
                S1 = int(L1 ** 0.5)
                S2 = int(L2 ** 0.5)
                absolute_pos_embed_pretrained = absolute_pos_embed_pretrained.reshape(-1, S1, S1, C1)
                absolute_pos_embed_pretrained = absolute_pos_embed_pretrained.permute(0, 3, 1, 2)
                absolute_pos_embed_pretrained_resized = torch.nn.functional.interpolate(
                    absolute_pos_embed_pretrained, size=(S2, S2), mode='bicubic')
                absolute_pos_embed_pretrained_resized = absolute_pos_embed_pretrained_resized.permute(0, 2, 3, 1)
                absolute_pos_embed_pretrained_resized = absolute_pos_embed_pretrained_resized.flatten(1, 2)
                state_dict[k] = absolute_pos_embed_pretrained_resized

    # check classifier, if not match, then re-init classifier to zero
    head_bias_pretrained = state_dict['classifier.bias']
    Nc1 = head_bias_pretrained.shape[0]
    Nc2 = model.classifier.bias.shape[0]
    if (Nc1 != Nc2):
        torch.nn.init.constant_(model.classifier.bias, 0.)
        torch.nn.init.constant_(model.classifier.weight, 0.)
        del state_dict['classifier.weight']
        del state_dict['classifier.bias']
        print(f"Error in loading classifier head, re-init classifier head to 0")

    msg = model.load_state_dict(state_dict, strict=False)
    print(msg)

    print(f"=> loaded successfully '{config.MODEL.PRETRAINED}'")

    torch.cuda.empty_cache()

In [151]:
swin_config = Box(swin_config)
load_pretrained(swin_config, model)

==============> Loading weight ../../models/swin2/simmim.pth for fine-tuning......
_IncompatibleKeys(missing_keys=['stages.0.blocks.0.attn_mask', 'stages.0.blocks.0.attn.relative_coords_table', 'stages.0.blocks.0.attn.relative_position_index', 'stages.0.blocks.1.attn_mask', 'stages.0.blocks.1.attn.relative_coords_table', 'stages.0.blocks.1.attn.relative_position_index', 'stages.1.blocks.0.attn_mask', 'stages.1.blocks.0.attn.relative_coords_table', 'stages.1.blocks.0.attn.relative_position_index', 'stages.1.blocks.1.attn_mask', 'stages.1.blocks.1.attn.relative_coords_table', 'stages.1.blocks.1.attn.relative_position_index', 'stages.2.blocks.0.attn_mask', 'stages.2.blocks.0.attn.relative_coords_table', 'stages.2.blocks.0.attn.relative_position_index', 'stages.2.blocks.1.attn_mask', 'stages.2.blocks.1.attn.relative_coords_table', 'stages.2.blocks.1.attn.relative_position_index', 'stages.2.blocks.2.attn_mask', 'stages.2.blocks.2.attn.relative_coords_table', 'stages.2.blocks.2.attn.relative

In [152]:
model.embeddings.patch_embeddings.weight[0]

tensor([[[ 0.0915,  0.0452,  0.0492, -0.1203],
         [-0.0344, -0.0366, -0.0396, -0.1072],
         [ 0.1185,  0.0418,  0.0388,  0.0307],
         [ 0.1307,  0.0662, -0.1047,  0.1280]],

        [[-0.1221, -0.0301, -0.1284,  0.0514],
         [-0.0583,  0.0590, -0.0052, -0.0822],
         [-0.0587, -0.0320,  0.0118, -0.0611],
         [-0.1188,  0.1018,  0.0837, -0.0119]],

        [[-0.0537, -0.0778, -0.0539,  0.1121],
         [ 0.0669, -0.0011,  0.1144,  0.0723],
         [ 0.0919, -0.0840, -0.0773,  0.0947],
         [-0.0562, -0.0688, -0.0784,  0.0850]]], grad_fn=<SelectBackward0>)

In [153]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [154]:
# Transforms 정의하기
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8,1), interpolation=transforms.InterpolationMode.LANCZOS),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.9, scale=(0.02, 0.33)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

data_dir = '../../data/sports'
batch_size = 960

train_path = data_dir+'/train'
valid_path = data_dir+'/valid'
test_path = data_dir+'/test'

# dataset load
train_data = ImageFolder(train_path, transform=train_transform)
valid_data = ImageFolder(valid_path, transform=test_transform)
test_data = ImageFolder(test_path, transform=test_transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [155]:
max_norm = 5.0

model.to(device)
model_path = '../../models/swin2/model_w_simmim.pth'

In [156]:
mixup_fn = Mixup(mixup_alpha=.8, 
                cutmix_alpha=.8, 
                prob=1., 
                switch_prob=0.5, 
                mode='batch',
                label_smoothing=.1,
                num_classes=100)

epochs = 100

criterion = nn.CrossEntropyLoss(label_smoothing=0.)
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=5e-2)
warmup_steps = int(len(train_loader)*(epochs+50)*0.1)
train_steps = len(train_loader)*(epochs+50)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer, 
                                                        num_warmup_steps=warmup_steps, 
                                                        num_training_steps=train_steps,
                                                        num_cycles=0.5)

In [157]:
training_time = 0
losses = []
val_losses = []
lrs = []
best_loss = float('inf')

# GradScaler 초기화
scaler = GradScaler()

for epoch in range(epochs):
    model.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for _, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs, labels = mixup_fn(inputs, labels)
        optimizer.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer)
        clip_grad_norm_(model.parameters(), max_norm=max_norm)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
            
        lr = optimizer.param_groups[0]["lr"]
        lrs.append(lr)
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)        

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in valid_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
    val_loss /= len(valid_loader)
    val_losses.append(val_loss)
    
    # 모델 저장
    if val_loss < best_loss:
        best_loss = val_loss
        vit_save = True
        if vit_save:
            torch.save(model.state_dict(), model_path)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
    
    if vit_save:
        text += f' - model saved!'
        vit_save = False

    print(text)
        
text = f"Epoch 당 평균 소요시간 : {training_time / epochs:.2f}초"      
print(text)

Epoch 1: 100%|██████████| 15/15 [00:50<00:00,  3.35s/it]


	Loss: 4.638499577840169, Val Loss: 4.54612922668457, LR: 6.666666666666667e-05, Duration: 51.56 sec - model saved!


Epoch 2: 100%|██████████| 15/15 [00:50<00:00,  3.36s/it]


	Loss: 4.5085446039835615, Val Loss: 4.290525436401367, LR: 0.00013333333333333334, Duration: 51.56 sec - model saved!


Epoch 3: 100%|██████████| 15/15 [00:49<00:00,  3.32s/it]


...

Epoch 10: 100%|██████████| 15/15 [00:50<00:00,  3.37s/it]


	Loss: 4.034962638219198, Val Loss: 3.284149646759033, LR: 0.0006666666666666666, Duration: 51.72 sec - model saved!


Epoch 11:  53%|█████▎    | 8/15 [00:31<00:27,  3.88s/it]

Unexpected exception formatting exception. Falling back to standard exception



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_4074161/3611770782.py", line 23, in <module>
    outputs = model(inputs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "/root/study/algorithm/04.Swin/model/swin_v2.py", line 514, in forward
    return x
  File "/root/study/algorithm/04.Swin/model/swin_v2.py", line 507, in forward_features
    x = self.pooler(x.transpose(1,2)) # (B, C, 1)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
# 예측 수행 및 레이블 저장
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 혼동 행렬 생성
cm = confusion_matrix(all_labels, all_preds)

# 예측과 실제 레이블
y_true = all_labels  # 실제 레이블
y_pred = all_preds  # 모델에 의해 예측된 레이블

# 전체 데이터셋에 대한 정확도
accuracy = accuracy_score(y_true, y_pred)

# 평균 정밀도, 리콜, F1-Score ('weighted')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

# 판다스 데이터프레임으로 결과 정리
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1_score]
})

# 데이터프레임 출력
performance_metrics

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Metric,Value
0,Accuracy,0.780000
1,Precision,0.797008
2,Recall,0.780000
3,F1 Score,0.766774
